In [187]:
import cv2
from ipywidgets import Video
import ipywidgets as widgets
from IPython.display import display
import numpy as np

In [188]:
video_capture = cv2.VideoCapture('Chess_Data\\chess_easy1.mp4')

### Full detection

In [189]:
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    frame_with_boxes = frame.copy()
    total_area = frame.shape[0] * frame.shape[1]
    threshold_lower = 0
    threshold_upper = np.inf
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > threshold_lower and area<threshold_upper:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame_with_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Display the result
    cv2.imshow('Frame with Boxes', frame_with_boxes)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
video_capture.release()
cv2.destroyAllWindows()

In [190]:
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Use HoughLinesP to detect lines in the edges
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)

    # Draw the detected lines on the original frame
    frame_with_lines = frame.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display the frame with detected lines
    cv2.imshow('Frame with Lines', frame_with_lines)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
video_capture.release()
cv2.destroyAllWindows()

### Keypoint Detection

In [191]:
sift = cv2.SIFT_create()

while True:   
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sift_kp1, sift_desc1 = sift.detectAndCompute(gray, None)
    img_kp1 = cv2.drawKeypoints(
        frame, sift_kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT
    )

    cv2.imshow('Video Frame', img_kp1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

### Corner Detection

In [192]:
while True:   
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    dst = cv2.cornerHarris(gray, 3, 3, 0.04)
    # result is dilated for marking the corners, not important
    dst = cv2.dilate(dst, None)

    # Threshold for an optimal value, it may vary depending on the image.
    frame[dst > 0.01 * dst.max()] = [0, 0, 255]

    cv2.imshow('Video Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()